In [1]:
import matlab.engine
import numpy as np
import h5py
from pathlib import Path
from tqdm import tqdm

import LokiCaller as env


In [2]:

#### experimental data from CO 

data_dict = {}

input_name_h5 = ["Tgas", "Tw", "Tnw", "current", "pressure"] 


with h5py.File("Buffer_Data/Experimental_data_Paper.hdf5", "r") as file:
    keys_hdf5 = list(file.keys())
    
    print(keys_hdf5)
    
    for key in keys_hdf5:
        if key in input_name_h5:
            data_dict[key] = file[key][:]



print(data_dict.keys())
print("current: ", data_dict['current'])
print("Tgas: ", data_dict['Tgas'])
print("Tw: ", data_dict["Tw"] )
print("pressure: ", data_dict["pressure"])


data_dict["current"] = data_dict['current'] * 1e-3
data_dict["Tw"] = data_dict["Tw"] + 273.15


['CO', 'File_Data', 'N', 'O', 'Tgas', 'Tnw', 'Tw', 'current', 'gamma_exp', 'pressure']
dict_keys(['Tgas', 'Tnw', 'Tw', 'current', 'pressure'])
current:  [20. 20. 20. 20. 20. 20. 40. 40. 40. 40. 40. 40. 40. 40. 40. 20. 20. 20.
 20. 20. 20. 20. 20. 40. 40. 40. 40. 40. 40. 40. 40. 40. 20. 20. 20. 20.
 20. 20. 20. 20. 40. 40. 40. 40. 40. 40. 40. 40. 40. 20. 20. 20. 20. 20.
 20. 20. 20. 40. 40. 40. 40. 40. 40. 40. 40. 40.]
Tgas:  [286.65477619 297.22283451 307.19459972 337.77707816 355.9657592
 384.9990093  311.66909798 325.28166517 338.27561182 350.6724884
 379.19392335 404.4550872  446.5435068  506.41305    563.45371875
 309.05030825 318.50558216 327.44533736 335.8930249  355.01334541
 371.5712352  398.4654143  437.6272125  332.67671995 345.01181125
 356.92211575 368.4187777  395.42388049 420.0874976  463.0867899
 528.6492125  586.65006094 323.92320518 333.81151149 343.09123838
 351.7918244  371.20273873 387.6379912  413.4214208  449.51935
 346.61859939 358.87287997 370.7763961  382.33543

In [3]:

base_path = 'LoKI_v3/Code'
input_file = 'CO2_novib_28mar2025_mod.in'
# output_densities_file = 'sim1/chemFinalDensities.txt'
# output_feat_file = 'sim1/swarmParameters.txt'

eng = matlab.engine.start_matlab()
eng.cd(base_path, nargout=0)

In [4]:
x_names = ["gasPressure", "gasTemperature", "nearWallTemperature", "wallTemperature"]
buffer_names = ["pressure", "Tgas", "Tnw", "Tw"]

map_x_buffer = {x_names[i]:buffer_names[i] for i in range(len(x_names))}

print(map_x_buffer)

sim = env.PhysicalSimulator(eng, base_path, input_file, x_names)


{'gasPressure': 'pressure', 'gasTemperature': 'Tgas', 'nearWallTemperature': 'Tnw', 'wallTemperature': 'Tw'}


In [5]:

current_exp = data_dict["current"]

params_list = []
for i in range(len(current_exp)):
    vec = []
    for key in x_names:
        key_buffer = map_x_buffer[key]
        vec.append(data_dict[key_buffer][i])
    params_list.append(vec)

print(params_list)

[[0.4, 286.654776186, 262.53133733208, 253.14999999999998], [0.6, 297.222834506, 265.49039366168, 253.14999999999998], [0.8, 307.194599725, 268.28248792299996, 253.14999999999998], [1.5, 337.777078162499, 276.84558188536, 253.14999999999998], [2.0, 355.9657592, 281.93841257599996, 253.14999999999998], [3.0, 384.9990093, 290.067722604, 253.14999999999998], [0.4, 311.669097978, 269.53534743384, 253.14999999999998], [0.6, 325.281665174, 273.34686624872, 253.14999999999998], [0.8, 338.275611821, 276.98517130988, 253.14999999999998], [1.0, 350.6724884, 280.456296752, 253.14999999999998], [1.5, 379.19392335, 288.442298538, 253.14999999999998], [2.0, 404.4550872, 295.515424416, 253.14999999999998], [3.0, 446.5435068, 307.300181904, 253.14999999999998], [5.0, 506.41305, 324.063654, 253.14999999999998], [7.5, 563.45371875, 340.03504125, 253.14999999999998], [0.4, 309.05030825, 286.80208631, 278.15], [0.6, 318.505582162, 289.44956300536, 278.15], [0.8, 327.445337357, 291.95269445996, 278.15], [1

In [6]:
print(len(current_exp))

66


In [ ]:

results = []

ne_min = 3e15
ne_max = 6e15


for counter in range(40, len(current_exp)):

    add_folder = {"folder": f"sim_{counter}"}

    params = params_list[counter]
    current_val = current_exp[counter]

    print(params)
    
    results_den, names_den = sim.solver_one_point(params, current_val, ne_min, ne_max, add_folder)
    results.append(results_den)
    print()



[0.4, 323.923205178, 305.36649744984, 298.15]
folder:  sim_32
electron_density: 3.0000000000000004
current_exp: 0.02
Call: 1 I: 0.016831340938513236 I_exp: 0.02
electron_density: 8.0
current_exp: 0.02
Call: 2 I: 0.049177506553436265 I_exp: 0.02
electron_density: 5.5
current_exp: 0.02
Call: 3 I: 0.032646716119423415 I_exp: 0.02
electron_density: 3.5098806307402555
current_exp: 0.02
Call: 4 I: 0.01998188117832947 I_exp: 0.02

[0.6, 333.811511494, 308.13522321832, 298.15]
folder:  sim_33
electron_density: 3.0000000000000004
current_exp: 0.02
Call: 5 I: 0.016142385106953924 I_exp: 0.02
electron_density: 8.0
current_exp: 0.02
Call: 6 I: 0.04304636028521046 I_exp: 0.02
electron_density: 5.5
current_exp: 0.02
Call: 7 I: 0.031190979527670557 I_exp: 0.02
electron_density: 3.58813520077159
current_exp: 0.02
Call: 8 I: 0.019590388549874573 I_exp: 0.02

[0.8, 343.091238381, 310.73354674667996, 298.15]
folder:  sim_34
electron_density: 3.0000000000000004
current_exp: 0.02
Call: 9 I: 0.0156803954654

In [ ]:


# #### write the results 
# with h5py.File("Buffer_Data/Experimental_data_Paper_LoKI.hdf5", "w") as file:
#     dt = h5py.string_dtype(encoding='utf-8')
#     file.create_dataset("LoKI_data", data=np.array(results))
#     file.create_dataset("LoKI_names", data=names_den)
